# Task 7: AutoFeatureSelector Tool
## This task is to test your understanding of various Feature Selection methods outlined in the lecture and the ability to apply this knowledge in a real-world dataset to select best features and also to build an automated feature selection tool as your toolkit

### Use your knowledge of different feature selector methods to build an Automatic Feature Selection tool
- Pearson Correlation
- Chi-Square
- RFE
- Embedded
- Tree (Random Forest)
- Tree (Light GBM)

### Dataset: FIFA 19 Player Skills
#### Attributes: FIFA 2019 players attributes like Age, Nationality, Overall, Potential, Club, Value, Wage, Preferred Foot, International Reputation, Weak Foot, Skill Moves, Work Rate, Position, Jersey Number, Joined, Loaned From, Contract Valid Until, Height, Weight, LS, ST, RS, LW, LF, CF, RF, RW, LAM, CAM, RAM, LM, LCM, CM, RCM, RM, LWB, LDM, CDM, RDM, RWB, LB, LCB, CB, RCB, RB, Crossing, Finishing, Heading, Accuracy, ShortPassing, Volleys, Dribbling, Curve, FKAccuracy, LongPassing, BallControl, Acceleration, SprintSpeed, Agility, Reactions, Balance, ShotPower, Jumping, Stamina, Strength, LongShots, Aggression, Interceptions, Positioning, Vision, Penalties, Composure, Marking, StandingTackle, SlidingTackle, GKDiving, GKHandling, GKKicking, GKPositioning, GKReflexes, and Release Clause.

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as ss
from collections import Counter
import math
from scipy import stats

In [2]:
player_df = pd.read_csv("data/fifa19.csv")

In [3]:
numcols = ['Overall', 'Crossing','Finishing',  'ShortPassing',  'Dribbling','LongPassing', 'BallControl', 'Acceleration','SprintSpeed', 'Agility',  'Stamina','Volleys','FKAccuracy','Reactions','Balance','ShotPower','Strength','LongShots','Aggression','Interceptions']
catcols = ['Preferred Foot','Position','Body Type','Nationality','Weak Foot']

In [4]:
player_df = player_df[numcols+catcols]

In [5]:
traindf = pd.concat([player_df[numcols], pd.get_dummies(player_df[catcols])],axis=1)
features = traindf.columns

traindf = traindf.dropna()

In [6]:
traindf = pd.DataFrame(traindf,columns=features)

In [7]:
y = traindf['Overall']>=87
X = traindf.copy()
del X['Overall']

In [8]:
X.head()

,Crossing,Finishing,ShortPassing,Dribbling,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Stamina,...,Nationality_Uganda,Nationality_Ukraine,Nationality_United Arab Emirates,Nationality_United States,Nationality_Uruguay,Nationality_Uzbekistan,Nationality_Venezuela,Nationality_Wales,Nationality_Zambia,Nationality_Zimbabwe
0,84.0,95.0,90.0,97.0,87.0,96.0,91.0,86.0,91.0,72.0,...,0,0,0,0,0,0,0,0,0,0
1,84.0,94.0,81.0,88.0,77.0,94.0,89.0,91.0,87.0,88.0,...,0,0,0,0,0,0,0,0,0,0
2,79.0,87.0,84.0,96.0,78.0,95.0,94.0,90.0,96.0,81.0,...,0,0,0,0,0,0,0,0,0,0
3,17.0,13.0,50.0,18.0,51.0,42.0,57.0,58.0,60.0,43.0,...,0,0,0,0,0,0,0,0,0,0
4,93.0,82.0,92.0,86.0,91.0,91.0,78.0,76.0,79.0,90.0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
len(X.columns)

223

### Set some fixed set of features

In [10]:
feature_name = list(X.columns)
# no of maximum features we need to select
num_feats=30

## Filter Feature Selection - Pearson Correlation

### Pearson Correlation function

In [11]:
def cor_selector(X, y,num_feats):
    # Your code goes here (Multiple lines)
    corelated_feat_list = []
    feature_list = X.columns.tolist()
    
    for i in X.columns.tolist():
        corelation = np.corrcoef(X[i], y)[0, 1]
        corelated_feat_list.append(corelation)
    
    corelated_feat_list = [0 if np.isnan(i) else i for i in corelated_feat_list]
    
    cor_feature = X.iloc[:,np.argsort(np.abs(corelated_feat_list))[-num_feats:]].columns.tolist()
    
    cor_support = [True if i in cor_feature else False for i in feature_list]
    # Your code ends here
    return cor_support, cor_feature

In [12]:
cor_support, cor_feature = cor_selector(X, y,num_feats)
print(str(len(cor_feature)), 'selected features')

30 selected features


### List the selected features from Pearson Correlation

In [13]:
cor_feature

['Nationality_Costa Rica',
 'Position_LAM',
 'Nationality_Uruguay',
 'Acceleration',
 'SprintSpeed',
 'Strength',
 'Nationality_Gabon',
 'Nationality_Slovenia',
 'Stamina',
 'Weak Foot',
 'Agility',
 'Crossing',
 'Nationality_Belgium',
 'Dribbling',
 'ShotPower',
 'LongShots',
 'Finishing',
 'BallControl',
 'FKAccuracy',
 'LongPassing',
 'Volleys',
 'ShortPassing',
 'Position_RF',
 'Position_LF',
 'Body Type_Courtois',
 'Body Type_PLAYER_BODY_TYPE_25',
 'Body Type_Neymar',
 'Body Type_Messi',
 'Body Type_C. Ronaldo',
 'Reactions']

## Filter Feature Selection - Chi-Sqaure

In [14]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer

### Chi-Squared Selector function

In [15]:
def chi_squared_selector(X, y, num_feats):
    # Your code goes here (Multiple lines)
    feature_list = X.columns.tolist()
    best_features = SelectKBest(score_func=chi2, k=num_feats)
    scaler = MinMaxScaler()
    chi_function = best_features.fit(scaler.fit_transform(X), y)
    chi_support = chi_function.get_support()
    chi_feature = X.loc[:, chi_support].columns.tolist()
    # Your code ends here
    return chi_support, chi_feature

In [16]:
chi_support, chi_feature = chi_squared_selector(X, y,num_feats)
print(str(len(chi_feature)), 'selected features')

30 selected features


### List the selected features from Chi-Square 

In [17]:
chi_feature

['Finishing',
 'ShortPassing',
 'LongPassing',
 'BallControl',
 'Volleys',
 'FKAccuracy',
 'Reactions',
 'LongShots',
 'Position_CM',
 'Position_LAM',
 'Position_LF',
 'Position_LW',
 'Position_RB',
 'Position_RF',
 'Body Type_C. Ronaldo',
 'Body Type_Courtois',
 'Body Type_Messi',
 'Body Type_Neymar',
 'Body Type_PLAYER_BODY_TYPE_25',
 'Nationality_Belgium',
 'Nationality_Costa Rica',
 'Nationality_Croatia',
 'Nationality_Egypt',
 'Nationality_England',
 'Nationality_France',
 'Nationality_Gabon',
 'Nationality_Slovakia',
 'Nationality_Slovenia',
 'Nationality_Spain',
 'Nationality_Uruguay']

## Wrapper Feature Selection - Recursive Feature Elimination

In [18]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler

### RFE Selector function

In [19]:
def rfe_selector(X, y, num_feats, estimator=LogisticRegression()):
    # Your code goes here (Multiple lines)
    scaler = MinMaxScaler()
    rfe = RFE(estimator=estimator, n_features_to_select=num_feats)
    rfe.fit(scaler.fit_transform(X), y)
    rfe_support = rfe.get_support()
    rfe_feature = X.loc[:, rfe_support].columns.tolist()
    # Your code ends here
    return rfe_support, rfe_feature

In [20]:
rfe_support, rfe_feature = rfe_selector(X, y,num_feats)
print(str(len(rfe_feature)), 'selected features')

30 selected features


### List the selected features from RFE

In [21]:
rfe_feature

['Finishing',
 'ShortPassing',
 'LongPassing',
 'BallControl',
 'Acceleration',
 'SprintSpeed',
 'Agility',
 'Volleys',
 'FKAccuracy',
 'Reactions',
 'Strength',
 'Weak Foot',
 'Position_CAM',
 'Position_CM',
 'Position_GK',
 'Position_LCB',
 'Position_LM',
 'Position_RB',
 'Position_RCB',
 'Position_RF',
 'Position_RM',
 'Position_RW',
 'Body Type_Courtois',
 'Body Type_PLAYER_BODY_TYPE_25',
 'Nationality_Belgium',
 'Nationality_Costa Rica',
 'Nationality_Gabon',
 'Nationality_Netherlands',
 'Nationality_Slovenia',
 'Nationality_Uruguay']

## Embedded Selection - Lasso: SelectFromModel

In [22]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler

In [23]:
def embedded_log_reg_selector(X, y, num_feats):
    # Your code goes here (Multiple lines)
    scaler = MinMaxScaler()
    estimator = LogisticRegression()
    embedded_logi_reg_selector = SelectFromModel(estimator, max_features=num_feats)
    embedded_logi_reg_selector = embedded_logi_reg_selector.fit(scaler.fit_transform(X), y)
    embedded_logi_reg_support = embedded_logi_reg_selector.get_support()
    embedded_logi_reg_feature = X.loc[:, embedded_logi_reg_support].columns.tolist()
    # Your code ends here
    return embedded_logi_reg_support, embedded_logi_reg_feature

In [24]:
embedded_lr_support, embedded_lr_feature = embedded_log_reg_selector(X, y, num_feats)
print(str(len(embedded_lr_feature)), 'selected features')

30 selected features


In [25]:
embedded_lr_feature

['Finishing',
 'ShortPassing',
 'LongPassing',
 'BallControl',
 'SprintSpeed',
 'Agility',
 'Volleys',
 'Reactions',
 'Strength',
 'Weak Foot',
 'Position_CAM',
 'Position_CM',
 'Position_GK',
 'Position_LCB',
 'Position_LM',
 'Position_RB',
 'Position_RCB',
 'Position_RM',
 'Position_RW',
 'Body Type_Courtois',
 'Body Type_Lean',
 'Body Type_Stocky',
 'Nationality_Belgium',
 'Nationality_Costa Rica',
 'Nationality_Croatia',
 'Nationality_France',
 'Nationality_Gabon',
 'Nationality_Netherlands',
 'Nationality_Slovenia',
 'Nationality_Uruguay']

## Tree based(Random Forest): SelectFromModel

In [26]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier

In [27]:
def embedded_rf_selector(X, y, num_feats):
    # Your code goes here (Multiple lines)
    scaler = MinMaxScaler()
    estimator = RandomForestClassifier(n_estimators=21, random_state=42)
    embedded_rf_selector = SelectFromModel(estimator, max_features=num_feats)
    embedded_rf_selector = embedded_rf_selector.fit(scaler.fit_transform(X), y)
    embedded_rf_support = embedded_rf_selector.get_support()
    embedded_rf_feature = X.loc[:, embedded_rf_support].columns.tolist()
    # Your code ends here
    return embedded_rf_support, embedded_rf_feature

In [28]:
embedded_rf_support, embedded_rf_feature = embedded_rf_selector(X, y, num_feats)
print(str(len(embedded_rf_feature)), 'selected features')

30 selected features


In [30]:
embedded_rf_feature

['Crossing',
 'Finishing',
 'ShortPassing',
 'Dribbling',
 'LongPassing',
 'BallControl',
 'Acceleration',
 'SprintSpeed',
 'Agility',
 'Stamina',
 'Volleys',
 'FKAccuracy',
 'Reactions',
 'Balance',
 'ShotPower',
 'Strength',
 'LongShots',
 'Aggression',
 'Interceptions',
 'Weak Foot',
 'Preferred Foot_Right',
 'Position_CB',
 'Position_LB',
 'Position_RCB',
 'Body Type_Courtois',
 'Body Type_Lean',
 'Body Type_PLAYER_BODY_TYPE_25',
 'Nationality_Slovenia',
 'Nationality_Spain',
 'Nationality_Uruguay']

## Tree based(Light GBM): SelectFromModel

In [31]:
from sklearn.feature_selection import SelectFromModel
from lightgbm import LGBMClassifier

In [32]:
def embedded_lgbm_selector(X, y, num_feats):
    # Your code goes here (Multiple lines)
    scaler = MinMaxScaler()
    estimator = LGBMClassifier(n_estimators=500,
                      learning_rate=0.05,
                      num_leaves=32,
                      colsample_bytree=0.2,
                      reg_alpha=3,
                      reg_lambda=1,
                      min_split_gain=0.01,
                      min_child_weight=40
    )
    embedded_lgbmc_selector = SelectFromModel(estimator, max_features=num_feats)
    embedded_lgbmc_selector = embedded_lgbmc_selector.fit(scaler.fit_transform(X), y)
    embedded_lgbmc_support = embedded_lgbmc_selector.get_support()
    embedded_lgbmc_feature = X.loc[:, embedded_lgbmc_support].columns.tolist()
    # Your code ends here
    return embedded_lgbmc_support, embedded_lgbmc_feature

In [33]:
embedded_lgbm_support, embedded_lgbm_feature = embedded_lgbm_selector(X, y, num_feats)
print(str(len(embedded_lgbm_feature)), 'selected features')

30 selected features


In [34]:
embedded_lgbm_feature

['Crossing',
 'Finishing',
 'ShortPassing',
 'Dribbling',
 'LongPassing',
 'BallControl',
 'Acceleration',
 'SprintSpeed',
 'Agility',
 'Stamina',
 'Volleys',
 'FKAccuracy',
 'Reactions',
 'Balance',
 'ShotPower',
 'Strength',
 'LongShots',
 'Aggression',
 'Interceptions',
 'Weak Foot',
 'Preferred Foot_Left',
 'Preferred Foot_Right',
 'Position_CAM',
 'Position_CB',
 'Position_CDM',
 'Position_CF',
 'Position_CM',
 'Position_GK',
 'Position_LAM',
 'Position_LB']

## Putting all of it together: AutoFeatureSelector Tool

In [37]:
pd.set_option('display.max_rows', None)
# put all selection together
feature_selection_df = pd.DataFrame({'Feature':feature_name, 'Pearson':cor_support, 'Chi-2':chi_support, 'RFE':rfe_support, 'Logistics':embedded_lr_support,
                                    'Random Forest':embedded_rf_support, 'LightGBM':embedded_lgbm_support})
# count the selected times for each feature
feature_selection_df['Total'] = np.sum(feature_selection_df, axis=1)
# display the top 100
feature_selection_df = feature_selection_df.sort_values(['Total','Feature'] , ascending=False)
feature_selection_df.index = range(1, len(feature_selection_df)+1)
feature_selection_df.head(num_feats)

C:\Users\prite\AppData\Roaming\Python\Python38\site-packages\numpy\core\fromnumeric.py:84: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  return reduction(axis=axis, out=out, **passkwargs)


,Feature,Pearson,Chi-2,RFE,Logistics,Random Forest,LightGBM,Total
1,Volleys,True,True,True,True,True,True,6
2,ShortPassing,True,True,True,True,True,True,6
3,Reactions,True,True,True,True,True,True,6
4,LongPassing,True,True,True,True,True,True,6
5,Finishing,True,True,True,True,True,True,6
6,BallControl,True,True,True,True,True,True,6
7,Weak Foot,True,False,True,True,True,True,5
8,Strength,True,False,True,True,True,True,5
9,SprintSpeed,True,False,True,True,True,True,5
10,Nationality_Uruguay,True,True,True,True,True,False,5


## Can you build a Python script that takes dataset and a list of different feature selection methods that you want to try and output the best (maximum votes) features from all methods?

In [63]:
def preprocess_dataset(dataset_path):
    # Your code starts here (Multiple lines)
    dataset = pd.read_csv(dataset_path)
    print("All the available feature are as below: \n")
    print(dataset.columns)
    print("\n\n")
    y_column = input("Please type which column would be target (y): ")
    
    y_threshold = int(input("Please type which value of 'y' should be True and False threshold? (Interger) : "))
    
    print("\nPredefined numerical columns are: \n", numcols)
    print("\nPredefined categorical columns are: \n", catcols)
   
    traindf = pd.concat([dataset[numcols], pd.get_dummies(dataset[catcols])],axis=1)
    features = traindf.columns
    
    # Dropping null values
    traindf = traindf.dropna()
    traindf = pd.DataFrame(traindf, columns=features)
    
    y = traindf[y_column]>=y_threshold
    X = traindf.copy()
    del X[y_column]
    
    # Setting number of features
    print(f"Out of all the columns, you need to select number of features that needs to be extracted, which should be lower than available number of columns. Total number available column is {len(dataset.columns)}.")
    num_feats = int(input("Please enter number of feature that need to be extracted (Integer): "))
    print(num_feats)
    
    # Your code ends here
    return X, y, num_feats

In [64]:
def autoFeatureSelector(dataset_path, methods=[]):
    # Parameters
    # data - dataset to be analyzed (csv file)
    # methods - various feature selection methods we outlined before, use them all here (list)
    
    # preprocessing
    X, y, num_feats = preprocess_dataset(dataset_path)
    
    # Run every method we outlined above from the methods list and collect returned best features from every method
    if 'pearson' in methods:
        print("\nApplying Pearson Correlation Filter")
        cor_support, cor_feature = cor_selector(X, y,num_feats)
    if 'chi-square' in methods:
        print("\nApplying Chi-Squared Filter")
        chi_support, chi_feature = chi_squared_selector(X, y,num_feats)
    if 'rfe' in methods:
        print("\nApplying Recursive Feature Elimination")
        rfe_support, rfe_feature = rfe_selector(X, y,num_feats)
    if 'log-reg' in methods:
        print("\nApplying Embedded Feature Selection using Logistic Regression")
        embedded_lr_support, embedded_lr_feature = embedded_log_reg_selector(X, y, num_feats)
    if 'rf' in methods:
        print("\nApplying Embedded Feature Selection using Random Forest")
        embedded_rf_support, embedded_rf_feature = embedded_rf_selector(X, y, num_feats)
    if 'lgbm' in methods:
        print("\nApplying Embedded Feature Selection using LGBM Classifier")
        embedded_lgbm_support, embedded_lgbm_feature = embedded_lgbm_selector(X, y, num_feats)
    
    
    # Combine all the above feature list and count the maximum set of features that got selected by all methods
    #### Your Code starts here (Multiple lines)
    pd.set_option('display.max_rows', None)
    # put all selection together
    feature_selection_df = pd.DataFrame({'Feature':feature_name, 'Pearson':cor_support, 'Chi-2':chi_support, 'RFE':rfe_support, 'Logistics':embedded_lr_support,
                                        'Random Forest':embedded_rf_support, 'LightGBM':embedded_lgbm_support})
    # count the selected times for each feature
    feature_selection_df['Total'] = np.sum(feature_selection_df, axis=1)
    # display the top 100
    feature_selection_df = feature_selection_df.sort_values(['Total','Feature'] , ascending=False)
    feature_selection_df.index = range(1, len(feature_selection_df)+1)
    best_features = feature_selection_df[:num_feats]["Feature"].values
    #### Your Code ends here
    return best_features

In [67]:
best_features = autoFeatureSelector(dataset_path="data/fifa19.csv", methods=['pearson', 'chi-square', 'rfe', 'log-reg', 'rf', 'lgbm'])
print("\nBest available features are: \n")
print(best_features)

All the available feature are as below: 

Index(['Unnamed: 0', 'ID', 'Name', 'Age', 'Photo', 'Nationality', 'Flag',
       'Overall', 'Potential', 'Club', 'Club Logo', 'Value', 'Wage', 'Special',
       'Preferred Foot', 'International Reputation', 'Weak Foot',
       'Skill Moves', 'Work Rate', 'Body Type', 'Real Face', 'Position',
       'Jersey Number', 'Joined', 'Loaned From', 'Contract Valid Until',
       'Height', 'Weight', 'LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW',
       'LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM',
       'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB', 'Crossing',
       'Finishing', 'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling',
       'Curve', 'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
   

Please type which column would be target (y):  Overall
Please type which value of 'y' should be True and False threshold? (Interger) :  90



Predefined numerical columns are: 
 ['Overall', 'Crossing', 'Finishing', 'ShortPassing', 'Dribbling', 'LongPassing', 'BallControl', 'Acceleration', 'SprintSpeed', 'Agility', 'Stamina', 'Volleys', 'FKAccuracy', 'Reactions', 'Balance', 'ShotPower', 'Strength', 'LongShots', 'Aggression', 'Interceptions']

Predefined categorical columns are: 
 ['Preferred Foot', 'Position', 'Body Type', 'Nationality', 'Weak Foot']
Out of all the columns, you need to select number of features that needs to be extracted, which should be lower than available number of columns. Total number available column is 89.


Please enter number of feature that need to be extracted (Integer):  10


10

Applying Pearson Correlation Filter

Applying Chi-Squared Filter

Applying Recursive Feature Elimination

Applying Embedded Feature Selection using Logistic Regression

Applying Embedded Feature Selection using Random Forest

Applying Embedded Feature Selection using LGBM Classifier

Best available features are: 

['ShortPassing' 'Reactions' 'Nationality_Uruguay' 'LongPassing'
 'FKAccuracy' 'BallControl' 'Agility' 'Volleys' 'Weak Foot' 'Position_RF']


C:\Users\prite\AppData\Roaming\Python\Python38\site-packages\numpy\core\fromnumeric.py:84: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  return reduction(axis=axis, out=out, **passkwargs)


### Last, Can you turn this notebook into a python script, run it and submit the python (.py) file that takes dataset and list of methods as inputs and outputs the best features